In [1]:
import pandas as pd
import numpy as np

In [32]:
#Read Data
uni = pd.read_csv("../../data/universities.csv")
prof = pd.read_csv("../../data/professors.csv", index_col=0)
article = pd.read_pickle("../../data/clean_articles.pkl")

#### Step 0: Calculate h-index and i10-index for professors

In [33]:
prof_citations = {row['id']: [] for idx, row in prof.iterrows()}

# Populate the citation counts for each professor
for idx, art in article.iterrows():
    cite_count = art['cite']
    authors = str(art['link_ids']).split(',')
    for author_id in authors:
        try:
            id = int(author_id)
            if id in prof_citations:
                prof_citations[id].append(cite_count)
        except:
            pass


In [34]:
def calculate_h_index(citations):
    citations.sort(reverse=True)
    h_index = 0
    for i, cite in enumerate(citations):
        if cite >= i + 1:
            h_index = i + 1
        else:
            break
    return h_index

def calculate_i10_index(citations):
    return sum(1 for cite in citations if cite >= 10)

# Calculate h-index and i10-index for each professor
results = []
for prof_id, citations in prof_citations.items():
    h_index = calculate_h_index(citations)
    i10_index = calculate_i10_index(citations)
    results.append({
        'professor_id': prof_id,
        'h_index': h_index,
        'i10_index': i10_index
    })

results_df = pd.DataFrame(results)

In [35]:
prof = prof.merge(results_df, left_on='id', right_on='professor_id', how='left')
prof.drop(columns=["professor_id"], inplace=True)

In [36]:
uni.is_governmental.fillna(0, inplace=True)
uni.is_governmental = uni.is_governmental.astype('b')

/tmp/ipykernel_17657/2543375143.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  uni.is_governmental.fillna(0, inplace=True)


#### Step 1: filter universities [keep govermental and non-medical universities]

In [37]:
uni = uni[uni.is_governmental == True]
uni_list = uni.University.to_list()
uni_list[:5]

['University of Tehran',
 'Sharif University of Technology',
 'Ferdowsi University of Mashhad',
 'Amirkabir University of Technology',
 'Shahid Beheshti University']

#### Step 2: filter professors [keep those who are in filtered universities - keep profs with citation more than 100]

In [41]:
del_prof = prof[prof["cited_by"] < 100]
del_prof_ids_1 = set(del_prof.id.to_list())

In [42]:
del_prof = prof[~ prof["university"].isin(uni_list)]
prof = prof[prof["university"].isin(uni_list)]


prof = prof[prof["cited_by"] >= 100]
prof

,id,name,university,user_id,affiliation,v_email_at,cited_by,intrests,department,h_index,i10_index
0,0,Ali Fahim,University of Tehran,_C4Iif8AAAAJ,"Assistant Professor of Physics, University of ...",ut.ac.ir,121695,"['Elementary Particle Physics', 'Physics of Co...",Physics,165,348
1,1,Mohammad Reza Ganjali,University of Tehran,IJ3XCecAAAAJ,"Professor of Analytical Chemistry, University ...",khayam.ut.ac.ir,49589,"['Sensor and biosensor', 'Elctroanalytical Che...",Chemistry,106,994
2,2,Norouzi Parviz,University of Tehran,usWI6hMAAAAJ,"Professor of Electrochemistry, Collage of Scie...",ut.ac.ir,24867,"['Electrochemistry', 'Electrochemical Sensors'...",Biology,80,520
3,3,Reza Tavakkoli-Moghaddam,University of Tehran,gqVUx4cAAAAJ,Distinguished Professor of Industrial Engineer...,ut.ac.ir,23651,"['Facilities design', 'Supply chain', 'Schedul...",History,76,479
4,4,Ali Akbar Moosavi-Movahedi,University of Tehran,sZysKwIAAAAJ,University of Tehran,ut.ac.ir,16918,['Biophysical Chemistry'],Chemistry,63,416
...,...,...,...,...,...,...,...,...,...,...,...
49372,49372,azizollah ardeshir-behrestaghi,Mazandaran University of Science and Technology,J0HyouoAAAAJ,Mazandaran university of science and technology,ustmb.ac.ir,400,[],unknown,9,9
49373,49373,Ali ‌Tajdin,Mazandaran University of Science and Technology,aTLGjt8AAAAJ,"Associate Professor of Industrial Engineering,...",ustmb.ac.ir,319,"['Operation Research', 'Fuzzy Logic', 'Statist...",Mathematics,8,6
49374,49374,Mahbod Armin,Mazandaran University of Science and Technology,2GTMJNoAAAAJ,Researcher at Mazandaran University of Science...,ustmb.ac.ir,244,"['Combustion Simulation & Engine\xa0…', 'Heat ...",Physics,6,5
49375,49375,Aydin Aghajani,Mazandaran University of Science and Technology,QhAn1zUAAAAJ,Mazandaran university of Science and technology,ustmb.ac.ir,201,"['Manufacturing systems', 'Reliability', 'Larg...",Mechanical Engineering,5,4


#### Step 3: filter article [keep (2020 to 2022 articles - filterd professors articles)]

In [43]:
article = article[(2020 <= article.year) & (article.year <= 2022)]

In [49]:
del_prof_ids = set(del_prof.id.to_list()).union(del_prof_ids_1)
del_prof_ids = set(map(str, del_prof_ids))

article = article.dropna(subset='link_ids')

for index, row in article.iterrows():
    link_ids = set(row["link_ids"].split(','))
    new_link_ids = link_ids - del_prof_ids
    article.at[index, "link_ids"] = ','.join(new_link_ids)

article = article.dropna(subset='link_ids')

#### Step 5: calculate Deapth and Breadth in feature extraction